In [1]:
import numpy as np
import random

# Galois Field Arithmetic

## Custom Functions

In [2]:
def ntp(num,field):
    '''
    Converts Number to Polynomial in the given Galois Field.
    '''
    return field.fetch_int(num)

    
def ptn(poly,field):
    '''
    Converts Polynomial in the given Galois Field to a Number
    '''
    #Gives the Polynomial Coefficients in Vector Form
    V = field.vector_space(map=False)
    
    #Instantiating given polynomial in Vector Form
    v = V(poly)
    
    #Convert the vector into a Bit string and convert to integer
    num = ''.join(map(str,v))[::-1]
    return int(num, 2)

## AES-128

In [3]:
# Instantiate Polynomial Ring in Galois Field
P.<b> = PolynomialRing(GF(2^8))

#Rijndael Irreducible Polynomial
R_p = b^8 + b^4 + b^3 + b + 1

#Instantiate the Galois Field with Rijndael Irreducible Polynomial as Modulus
aes_128.<x> = GF(2^8, modulus=R_p)

print(aes_128, type(aes_128), P, type(P), R_p, sep = '\n\n')

Finite Field in x of size 2^8

<class 'sage.rings.finite_rings.finite_field_givaro.FiniteField_givaro_with_category'>

Univariate Polynomial Ring in b over Finite Field in z8 of size 2^8

<class 'sage.rings.polynomial.polynomial_ring.PolynomialRing_dense_finite_field_with_category'>

b^8 + b^4 + b^3 + b + 1


In [4]:
# Creating to Galois Field Polynomials using Hexadecimal Notation
c_x = aes_128(ntp(0x02,aes_128))
d_x = aes_128(ntp(0xa3,aes_128))
type(c_x)

<class 'sage.rings.finite_rings.element_givaro.FiniteField_givaroElement'>

### Addition
- $c(X) = X = \texttt{02}$
- $d(X) = X^7 + X^5 + X+1= \texttt{a3}$

$$\begin{split}
\texttt{02} + \texttt{a3} &=X + (X^7 + X^5 + X+1)\\
&= X^7 + X^5 + 1 \\
&= \texttt{a1}
\end{split}$$

In [5]:
#Addition in Galois Field
print(c_x + d_x) 

x^7 + x^5 + 1


### Modular using Irreducible Polynomial

In [6]:
R_p

b^8 + b^4 + b^3 + b + 1

### Multiplication    
- $c(X) = X = \texttt{02}$
- $d(X) = X^7 + X^5 + X+1= \texttt{a3}$

$$\begin{split}
\texttt{02} \times \texttt{a3} &=X\times (X^7 + X^5 + X+1)\\
&= X^8 + X^6 + X^2+X \\
&= (R_p \times X^{8-8}) + X^8 + X^6 + X^2+X\\
&= (X^8 + X^4 + X^3 + X + 1) + (X^8 + X^6 + X^2+X)\\
&= X^6 + X^4 + X^3 + X^2 + 1\\
&= \texttt{5d}
\end{split}$$

In [7]:
#Direct Multiplication
prod = c_x * d_x
prod

x^6 + x^4 + x^3 + x^2 + 1

In [8]:
# Another Method
prod.mod(aes_128.modulus())

x^6 + x^4 + x^3 + x^2 + 1

### Lookup Table for Multiplication

In [9]:
def prod_table(field):
    '''
    Returns the Lookup Table for all the Multiplication in the given Galois Field
    '''
    n = field.order()
    
    #Initializing Table to Zero
    # Using * Operator twice would just created same referenced list causing issues
    prod_tbl = [[0]*n for i in range(n)]
    
    #Get all the numbers possible in the Field
    elements = []
    for i in range(n):
        elements.append(i)
        
    #Convert numbers to polynomials, galois field product and convert back to number
    for elem_1 in elements:
        for elem_2 in elements:
            prod_tbl[elem_1][elem_2] = ptn(ntp(elem_1,field)*ntp(elem_2,field), field)
            
    return prod_tbl

In [10]:
AES_128 = np.matrix(prod_table(aes_128))

In [11]:
print(AES_128)

[[  0   0   0 ...   0   0   0]
 [  0   1   2 ... 253 254 255]
 [  0   2   4 ... 225 231 229]
 ...
 [  0 253 225 ...  23  11 246]
 [  0 254 231 ...  11  18 236]
 [  0 255 229 ... 246 236  19]]


In [12]:
with open('Txt_Files/AES_128_prod.txt','wb') as f:
    for row in AES_128:
        np.savetxt(f, row, fmt='%#02x',delimiter=', ')

# AES-64

In [13]:
# Instantiate Polynomial Ring in Galois Field
R.<a> = PolynomialRing(GF(2^4))

#AES-64 Irreducible Polynomial
R_p = a^4 + a + 1

#Instantiate the Galois Field with AES-64 Irreducible Polynomial as Modulus
aes_64.<y> = GF(2^4, modulus=R_p)
print(aes_64, type(aes_64), R, type(R), sep = '\n\n')

Finite Field in y of size 2^4

<class 'sage.rings.finite_rings.finite_field_givaro.FiniteField_givaro_with_category'>

Univariate Polynomial Ring in a over Finite Field in z4 of size 2^4

<class 'sage.rings.polynomial.polynomial_ring.PolynomialRing_dense_finite_field_with_category'>


In [14]:
AES_64 = np.matrix(prod_table(aes_64))

In [15]:
with open('Txt_Files/AES_64_prod.txt','wb') as f:
    for row in AES_64:
        np.savetxt(f, row, fmt='%#01x',delimiter=', ')

## Mix Columns

In [16]:
def mix_col(field):
    '''
    Returns a Lookup Table containing Products of Elements of Galois Field with - 1,2,3
    '''
    n = field.order()
    
    #Initializing Table to Zero
    # Using * Operator twice would just created same referenced list causing issues
    prod_tbl = [[0]*n for i in range(3)]
    
    elements = [i for i in range(n)]
    
    #Multiply Mix Column Elements with all possible elements of field
    mix_elems = [1,2,3]
    for elem_1 in range(len(mix_elems)):
        for elem_2 in elements:
            prod_tbl[elem_1][elem_2] = ptn(ntp(mix_elems[elem_1],field)*ntp(elem_2,field),field)
            
    return prod_tbl


def inv_mix_col(field):
    '''
    Returns a Lookup Table containing Products of Elements of Galois Field with - 9, 11, 13, 14
    '''
    n = field.order()
    
    #Initializing Table to Zero
    # Using * Operator twice would just created same referenced list causing issues
    prod_tbl = [[0]*n for i in range(4)]
    
    elements = [i for i in range(n)]
    
    #Multiply Inverse Column Elements with all possible elements of field
    inv_mix_elems = [9,11,13,14]
    for elem_1 in range(len(inv_mix_elems)):
        for elem_2 in elements:
            prod_tbl[elem_1][elem_2] = ptn(ntp(inv_mix_elems[elem_1],field)*ntp(elem_2,field),field)
            
    return prod_tbl

In [17]:
#Storing in Text Files
AES_64_mix = np.matrix(mix_col(aes_64))
AES_64_inv_mix = np.matrix(inv_mix_col(aes_64))

AES_128_mix = np.matrix(mix_col(aes_128))
AES_128_inv_mix = np.matrix(inv_mix_col(aes_128))

with open('Txt_Files/AES_64_mix.txt','wb') as f:
    for row in AES_64_mix:
        np.savetxt(f, row, fmt='%#01x',delimiter=', ')

with open('Txt_Files/AES_64_inv_mix.txt','wb') as f:
    for row in AES_64_inv_mix:
        np.savetxt(f, row, fmt='%#01x',delimiter=', ')

with open('Txt_Files/AES_128_mix.txt','wb') as f:
    for row in AES_128_mix:
        np.savetxt(f, row, fmt='%#02x',delimiter=', ')

with open('Txt_Files/AES_128_inv_mix.txt','wb') as f:
    for row in AES_128_inv_mix:
        np.savetxt(f, row, fmt='%#02x',delimiter=', ')

## Round Constants

In [18]:
def r_cons(rounds,field):
    '''
    Returns all the Round Constant vectors for <rounds> number of rounds
    '''
    r_con_poly = [0]*rounds
    r_con_poly[0] = ntp(0x01,field)
    multiplier = ntp(0x02,field)
    for i in range(1,rounds):
         r_con_poly[i] = r_con_poly[i-1]*multiplier
    r_con = [[ptn(poly,field),0,0,0] for poly in r_con_poly]
    return r_con

In [19]:
#10 Round Round Constants for AES-64 and AES-128
AES_128_r_con_vec = np.matrix(r_cons(10,aes_128))
AES_64_r_con_vec = np.matrix(r_cons(10,aes_64))


#Storing in Text Files
with open('Txt_Files/AES_64_rcon.txt','wb') as f:
    for row in AES_64_r_con_vec:
        np.savetxt(f, row, fmt='%#01x',delimiter=', ')

with open('Txt_Files/AES_128_rcon.txt','wb') as f:
    for row in AES_128_r_con_vec:
        np.savetxt(f, row, fmt='%#02x',delimiter=', ')

# MD5 Collisions

In [20]:
import hashlib

#Make a List out of 10000 most used words in English
with open('Txt_Files/words.txt','rt') as f:
    words = [word[:-1] for word in f]

hashes = {}


def find_collisions(num_words, num_collisions):
    '''
    Find num_collisions amount of Single Collisions for a sentence of num_word words
    '''
    collisions = 0
    trial = 0
    while(collisions < num_collisions):
        #Building a sentence of num_word number of words
        sentence = " ".join(random.sample(words, num_words))
        #MD5 Hash of the sentence
        complete_hash = hashlib.md5(sentence.encode()).hexdigest()
        partial_hash = complete_hash[:4] + complete_hash[-4:]
        
        trial+=1
        
        if partial_hash not in hashes.keys():
            hashes[partial_hash] = sentence
        
        #Found Collision
        elif sentence != hashes[partial_hash]:
            collisions+=1
            print(f'Collision {collisions} after {trial} trials - for Partial Hash - {partial_hash}')
            print(f'{complete_hash} - {sentence}')
            print(f'{hashlib.md5(hashes[partial_hash].encode()).hexdigest()} - {hashes[partial_hash]}\n')
            trial = 0

In [21]:
find_collisions(4, 4)

Collision 1 after 104733 trials - for Partial Hash - 23e47f58
23e480a481c1e5f196460b767c017f58 - avon opportunity drinks products
23e4ef498b714e98c575c208a4427f58 - mw surf whats amanda

Collision 2 after 24069 trials - for Partial Hash - 17b51bee
17b5c1569659194467f292b0e5dd1bee - either needs receptors valve
17b549cb7b35d93fb1ce408e5af91bee - hans bristol fairfield cheaper

Collision 3 after 45404 trials - for Partial Hash - 7ecd4043
7ecdfaddd2a4cd6a53ae73050e844043 - deadly restricted steal hist
7ecd2df2763167c360ed18e9c23a4043 - measurements thumbs promote personal

Collision 4 after 1456 trials - for Partial Hash - 2e67511d
2e67a0cd31a4d7bdb54aed293b39511d - kay mug graduates albert
2e6760085aba1daae34d787249f2511d - floral steady hardcover hypothesis



# Understanding Cryptography - 11.8

In [22]:
def xor_hash(string):
    # Each character in 8 Bit Binary Format
    binary_format = [format(ord(char), '08b') for char in string]
    
    #XOR of each bit in every Byte
    x_hash = "".join([str(byte.count('1')%2) for byte in binary_format])
    
    return x_hash

def break_xor_hash(x_hash, num_preimage):
    '''
    Returns a dictionary containing num_preimage number of second preimages for the given Hash
    '''
    #Maximum 36 Preimages are possible from the words we chose.
    if(num_preimage > 36):
        raise ValueError(f"Cannot find {num_preimage} number of second preimages. Max limit is 36.\n")
    
    #Make a List of words with same length as given hash from the 10000 most used words in English
    with open('Txt_Files/words.txt','rt') as f:
        words = [word[:-1].upper() for word in f if len(word)==(len(x_hash)+1)]
    
    count = 0
    second_preimages = {}
    #Find num_preimages number of second preimages
    while(count < num_preimage):
        word = random.choice(words)
        
        if xor_hash(word) == x_hash and word not in second_preimages.keys() :
            second_preimages[word] = xor_hash(word)
            count+=1
            
    return second_preimages

In [23]:
xor_hash("CRYPTO")

'110011'

In [24]:
for char in range(65,91):
    print(f"{chr(char)}", end = " ")
    
print()

for char in range(65,91):
    print(f"{xor_hash(chr(char))}", end = " ")

A B C D E F G H I J K L M N O P Q R S T U V W X Y Z 
0 0 1 0 1 1 0 0 1 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0 0 

In [25]:
crypto_hash = xor_hash("CRYPTO")

#Find second preimages for CRYPTO word
second_preimages = break_xor_hash(crypto_hash, 36)

for word, word_hash in second_preimages.items():
    print(f"{word} - {word_hash}")

LEADER - 110011
WINNER - 110011
TRADER - 110011
ROBBIE - 110011
COUPLE - 110011
ETHNIC - 110011
TEMPLE - 110011
REPAIR - 110011
TENDER - 110011
WONDER - 110011
TRANCE - 110011
FRASER - 110011
LESSER - 110011
FEMALE - 110011
COPPER - 110011
RENDER - 110011
TOMATO - 110011
LENDER - 110011
LIABLE - 110011
READER - 110011
FINGER - 110011
LONGER - 110011
FISHER - 110011
CRADLE - 110011
COMMIT - 110011
FLAVOR - 110011
TIMBER - 110011
FOSSIL - 110011
WINDOW - 110011
REDUCE - 110011
FRANCE - 110011
FINDER - 110011
TRAVEL - 110011
RESULT - 110011
FIGURE - 110011
REBATE - 110011
